### Requirements

- httpx
- folium
- owslib

`!python -m pip install httpx folium owslib`

By default this notebook will assume you have titiler-stacfastpi application running locally 

```
uvicorn titiler.stacapi.main:app --port 8000
```

In [ ]:
import httpx

endpoint = "http://127.0.0.1:8000"

resp = httpx.get(endpoint)
print(resp)

In [ ]:
resp = httpx.get(
    f"{endpoint}/wmts",
    params={
        "service": "wmts",  # Mandatory
        "request": "GetCapabilities",  # Mandatory
        "version": "1.0.0",  # Optional
    },
)
print(resp)
print(resp.headers)

In [ ]:
print(resp.url)

In [ ]:
from owslib.wmts import WebMapTileService

wmts = WebMapTileService(
    f"{endpoint}/wmts?service=wmts&request=GetCapabilities&version=1.0.0",
)
print(wmts.identification)
print()
for layer in list(wmts.contents):
    print(layer)

print()
layer = wmts["MAXAR_Emilia_Romagna_Italy_flooding_may23_visual"]
print(layer)
print(layer.boundingBoxWGS84)
print()
print("Supported TMS:")
print(layer.tilematrixsetlinks)
print()
print("TMS Limits:")
print(layer.tilematrixsetlinks["WebMercatorQuad"].tilematrixlimits)
print()
print("Layer dimensions")
print(layer.dimensions)

In [ ]:
tile = wmts.gettile(
    layer="MAXAR_Emilia_Romagna_Italy_flooding_may23_visual",
    style="default",
    tilematrixset="WebMercatorQuad",
    tilematrix="13",
    column=4373,
    row=2963,
    format="image/png",
    time="2023-05-23",
)

In [ ]:
from IPython.display import Image

Image(tile.read())

In [ ]:
from folium import Map, TileLayer

m = Map(location=(44.52, 12.22), zoom_start=14)

layer = "MAXAR_Emilia_Romagna_Italy_flooding_may23"
TIME = "2023-05-23"
TileMatrixSet = "WebMercatorQuad"

tiles_url = f"http://127.0.0.1:8000/layers/{layer}/default/{TIME}/{TileMatrixSet}/{{z}}/{{x}}/{{y}}.png?assets=visual&asset_bidx=visual%7C1%2C2%2C3"

TileLayer(tiles=tiles_url, opacity=1, attr="DigitalGlobe OpenData").add_to(m)

m

In [ ]:
from folium import Map, TileLayer

col_map = Map(location=(44.52, 12.22), zoom_start=14)

collection_id = "MAXAR_Emilia_Romagna_Italy_flooding_may23"
TileMatrixSet = "WebMercatorQuad"

tiles_url = f"http://127.0.0.1:8000/collections/{collection_id}/tiles/{TileMatrixSet}/{{z}}/{{x}}/{{y}}.png?assets=visual&asset_bidx=visual%7C1%2C2%2C3"

TileLayer(tiles=tiles_url, opacity=1, attr="DigitalGlobe OpenData").add_to(col_map)

col_map

In [ ]:
from folium import Map, TileLayer

col_map = Map(location=(44.52, 12.22), zoom_start=14)

collection_id = "MAXAR_Emilia_Romagna_Italy_flooding_may23"
TileMatrixSet = "WebMercatorQuad"
date = "2023-04-23"

tiles_url = f"http://127.0.0.1:8000/collections/{collection_id}/tiles/{TileMatrixSet}/{{z}}/{{x}}/{{y}}.png?assets=visual&asset_bidx=visual%7C1%2C2%2C3&datetime={date}"

TileLayer(tiles=tiles_url, opacity=1, attr="DigitalGlobe OpenData").add_to(col_map)

col_map